# Notebook pour la Résolution de Sudoku avec DancingLinks

Ce notebook présentera deux approches pour résoudre des puzzles de Sudoku en utilisant l'algorithme Dancing Links (DLX). Nous explorerons d'abord une solution basée sur la bibliothèque `DlxLib`, puis une solution optimisée développée from scratch.

## 1. Introduction à Dancing Links

Dancing Links (DLX) est une technique efficace pour résoudre des problèmes de couverture exacte, popularisée par Donald Knuth. Elle est souvent utilisée pour des problèmes comme le Sudoku, où l'objectif est de couvrir toutes les contraintes avec un ensemble de solutions possibles.

**Références :**
- [Dancing Links Algorithm](https://en.wikipedia.org/wiki/Dancing_Links)
- [DLXLib Documentation](https://github.com/tomerfiliba/dlx)

### Théorie de la Couverture Exacte

Le problème de la couverture exacte consiste à couvrir un ensemble d'éléments, chaque élément étant couvert par exactement un sous-ensemble. Cela est utile pour des problèmes tels que le pavage d'un échiquier avec des pentominos, le problème des huit reines, et la résolution de Sudoku. Le problème est NP-complet et peut être résolu par l'algorithme X de Donald Knuth, souvent implémenté avec la technique des Dancing Links (DLX).
- [Problèmes de couverture exacte](https://fr.wikipedia.org/wiki/Probl%C3%A8me_de_la_couverture_exacte).

#### Exemples

Soit un ensemble U = {0, 1, 2, 3, 4} et une collection de sous-ensembles S = {E, I, P} avec E = {0, 2, 4}, I = {1, 3} et P = {2, 3}. Une couverture exacte de U est une sous-collection de S où chaque élément de U est contenu dans exactement un sous-ensemble de cette sous-collection, par exemple {E, I}.

#### Représentation Matricielle

Le problème de la couverture exacte peut être représenté par une matrice où chaque ligne représente un sous-ensemble et chaque colonne représente un élément. Une entrée de la matrice est 1 si l'élément de la colonne est dans le sous-ensemble de la ligne, et 0 sinon. Une couverture exacte est une sélection de lignes telle que chaque colonne contient exactement un 1.

Pour le Sudoku, la matrice contient 729 lignes (pour chaque cellule et chaque valeur possible) et 324 colonnes (pour les contraintes de ligne-colonne, ligne-nombre, colonne-nombre, et boîte-nombre).


## 2. Configuration de l'environnement

Installez les packages nécessaires pour ce notebook :

In [ ]:
#r "nuget: DlxLib"

### Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [ ]:
#!import Sudoku-0-Environment.ipynb

## 3. Implémentation avec DLXLib

Nous allons commencer par implémenter le solver en utilisant la bibliothèque `DlxLib`.

In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using DlxLib;

public class DancingLinkSolver : ISudokuSolver
{
     // Méthode principale pour résoudre un Sudoku
    public SudokuGrid Solve(SudokuGrid s)
    {
        // Conversion de la grille de Sudoku en une liste de tuples représentant les contraintes internes
        var internalRows = BuildInternalRowsForGrid(s);

        // Conversion des contraintes internes en lignes compatibles avec DLX
        var dlxRows = BuildDlxRows(internalRows);

        // Résolution du problème de couverture exacte avec DLX
        var solutions = new Dlx()
            .Solve(dlxRows, d => d, r => r)
            .ToImmutableList();

         // Conversion de la solution trouvée en une grille de Sudoku
        return SolutionToGrid(internalRows, solutions.First());
    }
    
    // Construction des contraintes internes pour chaque cellule de la grille
    private static IImmutableList<Tuple<int, int, int, bool>> BuildInternalRowsForGrid(SudokuGrid grid)
    {
        var internalRows = new List<Tuple<int, int, int, bool>>();
        for (int row = 0; row < 9; row++)
        {
            for (int col = 0; col < 9; col++)
            {
                int value = grid.Cells[row, col];
                internalRows.AddRange(BuildInternalRowsForCell(row, col, value));
            }
        }
        return internalRows.ToImmutableList();
    }

    // Construction des contraintes internes pour une cellule spécifique
    private static IImmutableList<Tuple<int, int, int, bool>> BuildInternalRowsForCell(int row, int col, int value)
    {
        if (value >= 1 && value <= 9)
        {
            return ImmutableList.Create(Tuple.Create(row, col, value, true));
        }
        else
        {
            var internalRows = new List<Tuple<int, int, int, bool>>(9);
            for (int v = 1; v <= 9; v++)
            {
                internalRows.Add(Tuple.Create(row, col, v, false));
            }
            return internalRows.ToImmutableList();
        }
    }

    // Conversion des contraintes internes en lignes pour DLX
    private static IImmutableList<IImmutableList<int>> BuildDlxRows(IEnumerable<Tuple<int, int, int, bool>> internalRows)
    {
        var dlxRows = new List<IImmutableList<int>>();
        foreach (var internalRow in internalRows)
        {
            dlxRows.Add(BuildDlxRow(internalRow));
        }
        return dlxRows.ToImmutableList();
    }

    // Construction d'une ligne DLX à partir d'une contrainte interne
    private static IImmutableList<int> BuildDlxRow(Tuple<int, int, int, bool> internalRow)
    {
        var row = internalRow.Item1;
        var col = internalRow.Item2;
        var value = internalRow.Item3;
        var box = RowColToBox(row, col);
        var result = new int[4 * 9 * 9];

        // Chaque contrainte est représentée par un 1 dans les colonnes appropriées
        result[row * 9 + col] = 1; 
        result[9 * 9 + row * 9 + value - 1] = 1;
        result[2 * 9 * 9 + col * 9 + value - 1] = 1;
        result[3 * 9 * 9 + box * 9 + value - 1] = 1;
        return result.ToImmutableList();
    }

    // Conversion des coordonnées de ligne et de colonne en un index de boîte
    private static int RowColToBox(int row, int col)
    {
        return row - (row % 3) + (col / 3);
    }

    // Conversion de la solution DLX en une grille de Sudoku
    private static SudokuGrid SolutionToGrid(
        IReadOnlyList<Tuple<int, int, int, bool>> internalRows,
        Solution solution)
    {
        var grid = new int[9, 9];
        foreach (var (row, col, value, _) in solution.RowIndexes.Select(rowIndex => internalRows[rowIndex]))
        {
            grid[row, col] = value;
        }
        return new SudokuGrid { Cells = grid };
    }
}

## 4. Implémentation Optimisée from Scratch

Nous allons maintenant implémenter une version optimisée de l'algorithme DLX.
Les Dancing Links (DLX) sont une technique pour ajouter et supprimer efficacement des nœuds d'une liste doublement chaînée circulaire. Utilisés pour implémenter l'algorithme X de Knuth, ils permettent de résoudre des problèmes de couverture exacte comme le Sudoku.

### Implémentation de l'Algorithme DLX

L'algorithme X est un algorithme de backtracking récursif qui trouve toutes les solutions au problème de couverture exacte. Pour améliorer l'efficacité, une matrice creuse est utilisée où seuls les 1 sont stockés.

### Fonctionnement

- Chaque nœud de la matrice pointe vers les nœuds adjacents à gauche et à droite (dans la même ligne), en haut et en bas (dans la même colonne), et vers l'en-tête de colonne.
- Chaque colonne a un nœud spécial "en-tête de colonne" inclus dans la liste circulaire des colonnes restantes.
- Lors de l'élimination d'une colonne, les lignes contenant un 1 dans cette colonne sont également éliminées, car elles entrent en conflit.

### Sélection et Couverture

- Sélectionnez une colonne avec le plus petit nombre de 1.
- Pour chaque ligne contenant un 1 dans cette colonne, ajoutez cette ligne à la solution partielle et éliminez les colonnes avec un 1 dans cette ligne.
- Répétez jusqu'à ce que toutes les colonnes soient éliminées, formant ainsi une solution.
- Pour revenir en arrière, restaurez les colonnes et les lignes éliminées dans l'ordre inverse.

In [ ]:
public class DlxCustomized
{
    // Classe représentant l'en-tête de colonne dans DLX
    class NodeHead : Node
    {
        internal int size;

        public NodeHead() : base(null) { }
    }

    // Classe représentant un nœud dans la structure DLX
    class Node
    {
        internal Node right = null;
        internal Node left = null;
        internal Node up = null;
        internal Node down = null;
        internal NodeHead nodeHead = null;
        
        internal int rowIndex;
        internal int column ;
        
        internal int value;

        public Node(NodeHead t)
        {
            nodeHead = t;
        }
    }
    
    private NodeHead root;
    private bool stop = false;
    private LinkedList<Node> solutions = new LinkedList<Node>();
    private SudokuGrid sudoku;
    
    // Constructeur initialisant la grille de Sudoku
    public DlxCustomized(SudokuGrid sudokuGrid)
    {
        sudoku = sudokuGrid;
        root = new NodeHead();
        root.right = root;
        root.left = root;
    }

    // Méthode principale pour résoudre le Sudoku
    public SudokuGrid Solve()
    {
        Init();
        search();
        foreach(Node node in solutions)
        {
            sudoku.Cells[node.rowIndex, node.column] = node.value;
        }
        return sudoku;
    }
    
    // Initialisation de la structure DLX
    public void Init()
    {
        Node[] tmp = new Node[9 * 9 * 4];
        root = new NodeHead();
        Node currentNode = root;
        
        for (int j = 0; j < 324; j++)
        {
            currentNode.right = new NodeHead();
            currentNode.right.left = currentNode;
            currentNode = currentNode.right;
            currentNode.rowIndex = j;
            currentNode.up = currentNode;
            currentNode.down = currentNode;
            tmp[j] = currentNode;
        }
        currentNode.right = root;
        root.left = currentNode;

        // Initialisation des nœuds pour chaque cellule du Sudoku

        int imatrix = 0;
        for (int i = 0; i < 9; i++)
        {
            for (int j = 0; j < 9; j++)
            {
                int value = sudoku.Cells[i ,j];
                Node tmpRCCNode, tmpRNCNode, tmpCNCNode, tmpBNCNode;
                
                if (value == 0)
                {
                    value = 1;
                    // Contrainte de cellule
                    int RCC = 9 * i + j; 
                    // Contrainte de ligne
                    int RNC = 81 + 9 * i + value - 1;
                    // Contrainte de colonne
                    int CNC = 162 + 9 * j + value - 1;
                    // Contrainte de bloc
                    int BNC = 243 + ((i / 3) * 3 + j / 3) * 9 + value - 1;
                    int end = imatrix + 9;
                    for (; imatrix < end; imatrix++)
                    {
                         // Création des nœuds pour les contraintes
                        tmpRCCNode = new Node((NodeHead)tmp[RCC].down);
                        tmpRNCNode = new Node((NodeHead)tmp[RNC].down);
                        tmpCNCNode = new Node((NodeHead)tmp[CNC].down);
                        tmpBNCNode = new Node((NodeHead)tmp[BNC].down);
                        
                        // Initialisation des nœuds avec les indices de ligne et de colonne
                        tmpRCCNode.rowIndex = i;
                        tmpRCCNode.column = j;
                        tmpRCCNode.value = value;
                        tmpRNCNode.rowIndex = i;
                        tmpRNCNode.column = j;
                        tmpRNCNode.value = value;
                        tmpCNCNode.rowIndex = i;
                        tmpCNCNode.column = j;
                        tmpCNCNode.value = value;
                        tmpBNCNode.rowIndex = i;
                        tmpBNCNode.column = j;
                        tmpBNCNode.value = value++;

                        // Mise à jour des tailles des en-têtes de colonne
                        ((NodeHead)tmp[RCC].down).size++;
                        ((NodeHead)tmp[RNC].down).size++;
                        ((NodeHead)tmp[CNC].down).size++;
                        ((NodeHead)tmp[BNC].down).size++;

                        // Liaisons des nœuds entre eux pour former des listes doublement chaînées circulaires
                        tmpRCCNode.right = tmpRNCNode;
                        tmpRNCNode.right = tmpCNCNode;
                        tmpCNCNode.right = tmpBNCNode;
                        tmpBNCNode.right = tmpRCCNode;
                        tmpBNCNode.left = tmpCNCNode;
                        tmpCNCNode.left = tmpRNCNode;
                        tmpRNCNode.left = tmpRCCNode;
                        tmpRCCNode.left = tmpBNCNode;

                        // Liaisons des nœuds avec leurs en-têtes de colonne respectifs
                        tmpRCCNode.up = tmp[RCC];
                        tmpRCCNode.down = tmp[RCC].down;
                        tmp[RCC].down = tmpRCCNode;
                        tmp[RCC] = tmpRCCNode;
                        tmpRNCNode.up = tmp[RNC];
                        tmpRNCNode.down = tmp[RNC].down;
                        tmp[RNC].down = tmpRNCNode;
                        tmp[RNC++] = tmpRNCNode;
                        tmpCNCNode.up = tmp[CNC];
                        tmpCNCNode.down = tmp[CNC].down;
                        tmp[CNC].down = tmpCNCNode;
                        tmp[CNC++] = tmpCNCNode;
                        tmpBNCNode.up = tmp[BNC];
                        tmpBNCNode.down = tmp[BNC].down;
                        tmp[BNC].down = tmpBNCNode;
                        tmp[BNC++] = tmpBNCNode;
                    }
                }
                else
                {
                     // Contrainte de cellule
                    int RCC = 9 * i + j;
                    // Contrainte de ligne
                    int RNC = 81 + 9 * i + value - 1;
                    // Contrainte de colonne
                    int CNC = 162 + 9 * j + value - 1;
                    // Contrainte de bloc
                    int BNC = 243 + ((i / 3) * 3 + j / 3) * 9 + value - 1;

                    // Création des nœuds pour les contraintes
                    tmpRCCNode = new Node((NodeHead)tmp[RCC].down);
                    tmpRNCNode = new Node((NodeHead)tmp[RNC].down);
                    tmpCNCNode = new Node((NodeHead)tmp[CNC].down);
                    tmpBNCNode = new Node((NodeHead)tmp[BNC].down);

                    // Initialisation des nœuds avec les indices de ligne et de colonne
                    tmpRCCNode.rowIndex = i;
                    tmpRCCNode.column = j;
                    tmpRCCNode.value = value;
                    tmpRNCNode.rowIndex = i;
                    tmpRNCNode.column = j;
                    tmpRNCNode.value = value;
                    tmpCNCNode.rowIndex = i;
                    tmpCNCNode.column = j;
                    tmpCNCNode.value = value;
                    tmpBNCNode.rowIndex = i;
                    tmpBNCNode.column = j;
                    tmpBNCNode.value = value;

                    // Liaisons des nœuds entre eux pour former des listes doublement chaînées circulaires
                    tmpRCCNode.right = tmpRNCNode;
                    tmpRNCNode.right = tmpCNCNode;
                    tmpCNCNode.right = tmpBNCNode;
                    tmpBNCNode.right = tmpRCCNode;
                    tmpBNCNode.left = tmpCNCNode;
                    tmpCNCNode.left = tmpRNCNode;
                    tmpRNCNode.left = tmpRCCNode;
                    tmpRCCNode.left = tmpBNCNode;

                     // Liaisons des nœuds avec leurs en-têtes de colonne respectifs
                    tmpRCCNode.up = tmp[RCC];
                    tmpRCCNode.down = tmp[RCC].down;
                    tmp[RCC].down = tmpRCCNode;
                    tmp[RCC] = tmpRCCNode;
                    tmpRNCNode.up = tmp[RNC];
                    tmpRNCNode.down = tmp[RNC].down;
                    tmp[RNC].down = tmpRNCNode;
                    tmp[RNC++] = tmpRNCNode;
                    tmpCNCNode.up = tmp[CNC];
                    tmpCNCNode.down = tmp[CNC].down;
                    tmp[CNC].down = tmpCNCNode;
                    tmp[CNC++] = tmpCNCNode;
                    tmpBNCNode.up = tmp[BNC];
                    tmpBNCNode.down = tmp[BNC].down;
                    tmp[BNC].down = tmpBNCNode;
                    tmp[BNC++] = tmpBNCNode;
                    imatrix++;
                }
            }
        }
    }
    
    // Méthode de recherche récursive
    public void search()
    {
        if (root.right == root)
        {
            stop = true;
            return;
        }

        NodeHead selected = (NodeHead)root.right;
        int c = selected.size;
        for (NodeHead currentNode = (NodeHead)root.right; currentNode != root; currentNode = (NodeHead)currentNode.right)
        {
            if (c > currentNode.size)
            {
                c = currentNode.size;
                selected = currentNode;
            }
        }

        cover(selected);

        for (Node iNode = selected.down; iNode != selected; iNode = iNode.down)
        {
            solutions.AddLast(iNode);
            for (Node jNode = iNode.right; jNode != iNode; jNode = jNode.right)
            {
                cover(jNode.nodeHead);
            }
            search();
            if (stop)
                return;
            solutions.RemoveLast();
            for (Node jNode = iNode.left; jNode != iNode; jNode = jNode.left)
            {
                uncover(jNode.nodeHead);
            }
        }
        uncover(selected);
        return;
    }

    // Méthode pour couvrir une colonne
    private void cover(NodeHead node)
    {
        node.left.right = node.right;
        node.right.left = node.left;
        for (Node iNode = node.down; iNode != node; iNode = iNode.down)
        {
            for (Node jNode = iNode.right; jNode != iNode; jNode = jNode.right)
            {
                jNode.up.down = jNode.down;
                jNode.down.up = jNode.up;
                jNode.nodeHead.size--;
            }
        }
    }

    // Méthode pour découvrir une colonne
    private void uncover(NodeHead node)
    {
        for (Node iNode = node.down; iNode != node; iNode = iNode.down)
        {
            for (Node jNode = iNode.right; jNode != iNode; jNode = jNode.right)
            {
                jNode.up.down = jNode;
                jNode.down.up = jNode;
                jNode.nodeHead.size++;
            }
        }
        node.left.right = node;
        node.right.left = node;
    }
}

### Implémentation du nouveau solver

In [ ]:
public class DancingLinkSolverWithCustomDlx : ISudokuSolver
{
    
       
    public SudokuGrid Solve(SudokuGrid sudoku)
    {
        DlxCustomized dlxCustomized = new DlxCustomized(sudoku);
        return dlxCustomized.Solve();
    }
}

## 5. Comparaison des Performances

Nous allons maintenant comparer les performances des deux approches sur des puzzles de différentes difficultés. Nous testerons les solveurs `Dancing Link Solver (DLXLib)` et `Dancing Link Solver (Customized)` sur des grilles de Sudoku de différentes difficultés (facile, moyenne, difficile).

### Résultats des Tests

Les temps d'exécution seront mesurés et comparés pour chaque solver et chaque niveau de difficulté.


In [ ]:
using System.Diagnostics;
using System.Threading;
using System.Threading.Tasks;

var solvers = new List<(string Name, ISudokuSolver Solver)>
{
    ("Dancing Link Solver (DLXLib)", new DancingLinkSolver()),
    ("Dancing Link Solver (Customized)", new DancingLinkSolverWithCustomDlx())
};

var results = SudokuHelper.TestSolvers(solvers);

// Affichage des résultats
foreach (var result in results)
{
    Console.WriteLine($"{result.SolverName} | Difficulty: {result.Difficulty} | Time: {result.Time} ms | Status: {result.Status}");
}

SudokuHelper.DisplayResults(results);